In [ ]:
#!pip install datascience
from datascience import *
import numpy as np

import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
%matplotlib inline

## Alameda County Juries

In [ ]:
# Data from an ACLU 2010 report
# Racial and Ethnic Disparities in Alameda County Jury Pools
# https://www.aclunc.org/sites/default/files/racial_and_ethnic_disparities_in_alameda_county_jury_pools.pdf

alameda = Table().with_columns(
    'Ethnicity', make_array('Asian', 'Black', 'Latino', 'White', 'Other'),
    'Eligible', make_array(0.15, 0.18, 0.12, 0.54, 0.01),
    'Panels', make_array(0.26, 0.08, 0.08, 0.54, 0.04)
)

alameda.set_format([1, 2], PercentFormatter(0))

In [ ]:
alameda.barh(0)

## Total Variation Distance

In [ ]:
diff = alameda.with_column('Difference',
                    alameda.column('Eligible') - alameda.column('Panels'))
diff

In [ ]:
abs_diff = diff.with_column('Abs. Difference',
                        np.abs(diff.column('Difference')))
abs_diff

In [ ]:
sum(abs_diff.column('Abs. Difference')) / 2

In [ ]:
def total_variation_distance(distribution_1, distribution_2):
    """Each distribution is an array of proportions that sums to 1."""
    return np.abs(distribution_1 - distribution_2).sum()/2

In [ ]:
def table_tvd(table, label_1, label_2):
    return total_variation_distance(table.column(label_1), table.column(label_2))

In [ ]:
observed = table_tvd(alameda, 'Eligible', 'Panels')
observed

## Simulating the statistic

In [ ]:
def get_one_simulated_panel(t, n):
    return t.select('Ethnicity').sample(n, weights=t.column('Eligible'))

get_one_simulated_panel(alameda, 1453)

In [ ]:
def simulate_once(t, n):
    simulated_panel = get_one_simulated_panel(t, n)
    counts = simulated_panel.group('Ethnicity')
    sim_proportions = counts.select('Ethnicity').with_column('Random',
                                    counts.column('count') / n)
    sim_proportions.set_format(1, PercentFormatter(0))
    return t.join('Ethnicity', sim_proportions)

In [ ]:
simulate_once(alameda, 1453)

In [ ]:
# Compute the empirical distribution of TVDs

tvds = make_array()

for i in np.arange(1000): # 1000 repetitions of the simulation
    sim_results = simulate_once(alameda, 1453)
    tvds = np.append(tvds, table_tvd(sim_results, 'Eligible', 'Random'))

results = Table().with_column('TVD', tvds)
results

In [ ]:
results.hist(bins=np.arange(0, 0.2, 0.005))

# Mendel

## 1. Select two hypotheses

* **Null:** For every plant, there is a 75% chance that it will have purple flowers, and a 25% chance that the flowers will be white, regardless of the colors in all the other plants.
* **Alternative:** Something else

In [ ]:
flowers = Table().with_columns(
    'Color', make_array('Purple', 'White'),
    'Model Proportion', make_array(0.75, 0.25),
    'Plants', make_array(705, 224)
)

flowers

In [ ]:
total_plants = flowers.column('Plants').sum()
total_plants

In [ ]:
observed_proportion = flowers.column('Plants').item(0)/total_plants
observed_proportion

## 2. Choose a test statistic

In [ ]:
observed_statistic = abs(observed_proportion - 0.75)
observed_statistic

## 3. Compute the Distribution of the Test Statistic under the Null Hypothesis

In [ ]:
model_colors = make_array('Purple', 'Purple', 'Purple', 'White')

repetitions = 5000

sampled_stats = make_array()

for i in np.arange(repetitions):
    new_sample = np.random.choice(model_colors, total_plants)
    proportion_purple = np.count_nonzero(new_sample == 'Purple')/total_plants
    sampled_stats = np.append(sampled_stats, abs(proportion_purple - 0.75))

results = Table().with_column('Distance from 0.75', sampled_stats)
results.hist()

## 4. Compare the Prediction to the Observed Data

In [ ]:
results.hist()

#Plot the observed statistic as a large red point on the horizontal axis
plots.scatter(observed_statistic, 0, color='red', s=30);

In [ ]:
# use p value for quantitative comparison 
# between observed statistic and null distribution
empirical_P = np.count_nonzero(sampled_stats >= observed_statistic)/repetitions
empirical_P

# Jelly beans

![jelly](https://imgs.xkcd.com/comics/significant.png)

In [ ]:
def test_stat(sample):
    return sum(sample=='Some') / len(sample)

def simulate_once(n_patients):
    acne_levels = make_array('Some', 'None')
    sample = np.random.choice(acne_levels, n_patients, p=make_array(.2, .8))
    return test_stat(sample)

In [ ]:
simulate_once(10)

In [ ]:
n = 1000
nrep = 10000
acne_fractions = make_array()
for i in range(nrep):
    acne_fractions = np.append(acne_fractions, simulate_once(n))
    
acne_fractions

In [ ]:
Table().with_column("Acne fraction", acne_fractions).hist()

In [ ]:
# for each color jellybean, count fraction of jellybean eaters with acne:
s = "A&W Cream Soda, A&W Root Beer, Berry Blue, Blueberry, Bubble Gum, Buttered Popcorn, Cantaloupe, Cappuccino, Caramel Corn, Chili Mango, Chocolate Pudding, Cinnamon, Coconut, Cotton Candy, Crushed Pineapple, Dr Pepper, French Vanilla, Green Apple, Island Punch, Juicy Pear, Kiwi, Lemon Drop, Lemon Lime, Sunkist Lemon, Licorice, Sunkist Lime, Mango, Margarita, Mixed Berry Smoothie, Orange Sherbet, Sunkist Orange, Peach, Sunkist Pink Grapefruit, Piña Colada, Plum, Pomegranate, Raspberry, Red Apple, Sizzling Cinnamon, Sour Cherry, Strawberry Cheesecake, Strawberry Daiquiri, Strawberry Jam, Sunkist Tangerine, Toasted Marshmallow, Top Banana, Tutti-Fruitti, Very Cherry, Watermelon, Wild Blackberry"
flavors = s.split(",")
# (there are 50 flavors)

for jelly in flavors:
    observed_statistic = simulate_once(1000)

    # and compute the P-value (an approximation based on the simulation)
    pval = sum(acne_fractions>=observed_statistic) / len(acne_fractions)
    if pval < .05:
        print(observed_statistic*100,"% of people eating", jelly, "jelly bean had acne")
        print("P value is", pval)
        print("Publish!", jelly, "flavor jelly causes acne.")
    else:
        continue